In [ ]:
import pandas as pd
from datasets import load_dataset
import os

In [2]:
import json
with open('./data/task170_hotpotqa_answer_generation.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [ ]:
data = data["Instances"]

In [ ]:
for i, example in enumerate(data):
    # Extract the input text and output text from the example
    input_text = example['input']
    output_text = example['output']
    
    # Split the input text into context and question parts based on "Question:"
    context_part, question_part = input_text.rsplit("Question:",1)
    example['context'] = context_part
    example['question'] = question_part

In [5]:
df = pd.DataFrame(data)
df = df.drop([4906]) #one bad sample causing error
df.shape

(6499, 5)

In [9]:
from datasets import Dataset
# Function to evaluate as Llama index does not support async evaluation for HFInference API

def generate_responses(test_questions, pdf_file, test_answers):
  outputs = [chat_bot.generate_response(q, pdf_file) for q in test_questions]

  answers = []
  contexts = []
  for output in outputs:
    answers.append(output['answer'])
    contexts.append([doc.page_content for doc in output['source_documents']])
  dataset_dict = {
        "question": test_questions,
        "answer": answers,
        "contexts": contexts,
  }
  if test_answers is not None:
    dataset_dict["ground_truth"] = test_answers
  #ds = Dataset.from_dict(dataset_dict)
  return dataset_dict

In [ ]:
import json
# from src import chatbot
import sys
sys.path.append('d:/Local-RAG-Assistant-Chatbot/') # change to your own path
from src import chatbot_base

import time
path =  "D:/Local-RAG-Assistant-Chatbot/config.yaml" # change to your own path
pdf_dir = "./merged_pdfs/"
chat_bot = chatbot_base.PDFChatBot(config_path=path)

# restart kernel and rerun with i = 10, i = 20, i = 30, i = 40, consecutive pdfs are not supported in a single run yet
i = 0
questions = df.loc[i:i+9]['question'].values.tolist()
test_answers = df.loc[i:i+9]['output'].values.tolist()
test_answers = [item for sublist in test_answers for item in sublist]
dataset_dict = generate_responses(questions, f"{pdf_dir}context_{i}_{i+9}.pdf", test_answers)

output_path = "./outputs/"
if not os.path.exists(output_path):
    os.makedirs(output_path)
output_file_path = output_path + f"outputs_base_hotpot_{i}_{i+9}.json"
with open(output_file_path, 'w') as json_file:
    json.dump(dataset_dict, json_file, indent=4)

